<a href="https://colab.research.google.com/github/anjanadevi05/Gamification_Fin_Edu/blob/main/financial_chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install pyspellchecker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 11.7 MB/s eta 0:00:00


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 58.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.3/316.3 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 73.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.8 MB/s et

In [ ]:
from transformers import pipeline
from spellchecker import SpellChecker
import pandas as pd
import re
import gradio as gr

In [ ]:
# Load your financial terms dataset
df = pd.read_csv('/content/FinRAD final.csv', encoding='latin1')
df = df[df['terms'].apply(lambda x: len(str(x)) > 1)]
df['clean_terms'] = df['terms'].apply(lambda x: re.sub(r'\([^)]*\)', '', str(x)).strip())
financial_data = dict(zip(df['terms'].str.lower(), df['definitions']))


In [ ]:
# Initialize the spell checker
spell = SpellChecker()

# Replace 'bert-base-uncased' with the specific model you want to use
nlp = pipeline('question-answering', model='bert-base-uncased', tokenizer='bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def get_definitions(question):
    # Check for exact matches first
    if isinstance(question, str) and question.lower() in financial_data:
        return [financial_data[question.lower()]]

    # Check if the question contains any financial terms
    matched_terms = [term for term in financial_data.keys() if isinstance(term, str) and term.lower() in question.lower()]

    if not matched_terms:
        # If no direct match, check for spelling mistakes and suggest corrections
        misspelled_words = spell.unknown(question.split())
        if misspelled_words:
            corrections = {word: spell.correction(word) for word in misspelled_words}
            suggestion = f"Did you mean: {', '.join(f'{correction}' for original, correction in corrections.items())}?"
            return [suggestion]
        else:
            return ["I'm sorry, I'm not sure if the entered term is a financial term"]

    # Return the definitions of all matched terms
    answers = []
    for term in matched_terms:
        answer = financial_data[term]
        if "refer" in answer.lower() or "See" in answer.lower():
            related_term = answer.lower().replace("refer ", "").replace("See ", "")
            if related_term in financial_data:
                answer = financial_data[related_term]
        answers.append(answer)

    return answers

In [ ]:
print("HI I'm your financial advisor!!!\n")
user_choice = "yes"

while user_choice.lower() != 'no':
    user_input = input("\nWhat specific term would you like assistance with?")
    responses = get_definitions(user_input)
    print("\n",user_input,": ")
    for response in responses:
        sentences = response.split('. ')  # Split the response into sentences
        for sentence in sentences:
            print(sentence)
            if sentence and sentence[-1] != '.':
                print()
    print("Hopefully,your doubt has been resolved!!!\n")
    user_choice = input("Is there another term you would like assistance with?(yes/no)")

print("\nThank you!! Have a great day!! Feel free to ask any questions.\n")

HI I'm your financial advisor!!!


What specific term would you like assistance with?eroi

 eroi : 
See RETURN ON INVESTMENT.
Hopefully,your doubt has been resolved!!!

Is there another term you would like assistance with?(yes/no)no

Thank you!! Have a great day!! Feel free to ask any questions.



In [ ]:
def chatbot_interface(user_input):
    responses = get_definitions(user_input)
    chat_history = "\n".join(responses)
    msg="If there is any another term you would like assistance with,enter in the input terminal\nThank you!! Have a great day!! Feel free to ask any questions.😊"
    res=f"{user_input}: {chat_history}\n\nHopefully, your doubt has been resolved!\n\n"
    return res,msg


face=gr.Interface(fn=chatbot_interface, inputs=gr.Textbox(label=" What specific term would you like assistance with?"),
                  outputs=[ gr.Textbox(label="RESPONSE OF FINANCIAL ADVISOR:"),
                           gr.Textbox(label="Chat Bot:"),],
                  title="Financial Advisor",
                 description="HI I'm your financial advisor!!!",)

face.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://8f4b46c34731f1ef22.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
test_data = [
    {"question": "capital lease", "expected_answer": "long-term, noncancelable lease (cf. operating lease)."},
    {"question": "Energy Return on Investment", "expected_answer": "Energy Return on Investment (EROI) is the ratio of the amount of usable energy obtained from a resource to the amount of energy expended to produce that net amount of energy. For instance it takes energy to locate, extract, deliver, and refine crude oil before it can be used for energy."},
    {"question": "credit card", "expected_answer": "A credit card is a thin rectangular piece of plastic or metal issued by a bank or financial services company, that allows cardholders to borrow funds with which to pay for goods and services with merchants that accept cards for payment. Credit cards impose the condition that cardholders pay back the borrowed money, plus any applicable interest, as well as any additional agreed-upon charges, either in full by the billing date or over time. An example of a credit card is the Chase Sapphire Reserve. (You can read our Chase Sapphire Reserve credit card review to get a good sense of all the various attributes of a credit card)."},
    # Add more test cases
]


In [ ]:
def evaluate_chatbot(test_data):
    correct_answers = 0
    total_questions = len(test_data)

    for data in test_data:
        question = data['question']
        expected_answer = data['expected_answer']

        responses = get_definitions(question)
        predicted_answer = responses[0] if responses else ""

        # Calculate precision, recall, and F1 if you have more complex evaluation needs
        if expected_answer.lower() == predicted_answer.lower():
            correct_answers += 1

    accuracy = correct_answers / total_questions
    return accuracy

# Run the evaluation
accuracy = evaluate_chatbot(test_data)
print(f"Chatbot Accuracy: {accuracy * 100:.2f}%")


Chatbot Accuracy: 0.00%


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming binary relevance for simplicity
y_true = [data['expected_answer'] for data in test_data]
y_pred = [get_definitions(data['question'])[0] for data in test_data]

# Convert to binary (1 if correct, 0 if incorrect)
binary_y_true = [1] * len(y_true)
binary_y_pred = [1 if pred.lower() == true.lower() else 0 for pred, true in zip(y_pred, y_true)]

precision = precision_score(binary_y_true, binary_y_pred)
recall = recall_score(binary_y_true, binary_y_pred)
f1 = f1_score(binary_y_true, binary_y_pred)

print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")


Precision: 0.00
Recall: 0.00
F1 Score: 0.00


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
for data in test_data:
    question = data['question']
    expected_answer = data['expected_answer']

    responses = get_definitions(question)
    predicted_answer = responses[0] if responses else ""

    print(f"Question: {question}")
    print(f"Expected Answer: {expected_answer}")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Match: {expected_answer.lower() == predicted_answer.lower()}")
    print("-" * 50)


Question: capital lease
Expected Answer: long-term, noncancelable lease (cf. operating lease).
Predicted Answer: A long-term LEASE CONTRACT, structured in con- ventional or LEVERAGED form, where the LESSEE accepts most/all of the RISKS and benefits of the leased property. Since the risks and economics aretransferable, ACCOUNTING rules generally require capital leases to be reflected on the corporate BALANCE SHEET. A capital lease is typically not cancella- ble, unless the lessee pays the LESSOR for any resulting losses. Also known as FINANCIAL LEASE. See also LEVERAGED LEASE, OPERATING LEASE.
Match: False
--------------------------------------------------
Question: Energy Return on Investment
Expected Answer: Energy Return on Investment (EROI) is the ratio of the amount of usable energy obtained from a resource to the amount of energy expended to produce that net amount of energy. For instance it takes energy to locate, extract, deliver, and refine crude oil before it can be used for e